# Import libs, mount drive

In [1]:
from google.colab import drive 
drive.mount('/content/drive')  #brings google drive files to be accessible under /content/

Mounted at /content/drive


In [4]:
%%capture
# fixes "NotImplementedError: Cannot convert a symbolic Tensor (Inputs/minibatch_gpu_in:0) to a numpy array." during run_training.py
!pip install numpy==1.19.5  

import os
import pickle
import numpy as np
import PIL.Image
import scipy
# %tensorflow_version 1.x
import tensorflow as tf


In [6]:
%%capture
# !git clone https://github.com/NVlabs/stylegan2.git
!git clone https://github.com/taziksh/stylegan2  #same as https://github.com/ZKTKZ/stylegan2.git
%cd /content/stylegan2
# import dnnlib
# import dnnlib.tflib as tflib

!nvcc test_nvcc.cu -o test_nvcc -run
!nvcc /content/stylegan2/test_nvcc.cu -o /content/stylegan2/test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

!pip install tensorboard



---
# Loading data into tfrecords





**Loading a subset of high-res faces**

https://www.kaggle.com/datasets/subinium/highresolution-anime-face-dataset-512x512

In [ ]:
# %%capture
# ! pip install kaggle
# ! mkdir ~/.kaggle
# # ! cp /content/drive/my\ Drive/kaggle.json ~/.kaggle/
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download subinium/highresolution-anime-face-dataset-512x512

Unzip only 5000 files from zip

https://stackoverflow.com/questions/22243031/unzip-only-limited-number-of-files-in-linux

In [ ]:
# !unzip -Z1 highresolution-anime-face-dataset-512x512.zip | head -5000 | sed 's| |\\ |g' | xargs unzip highresolution-anime-face-dataset-512x512.zip



---


**Download high-res directly from drive**

Takes 1min to unzip 5000 imgs, then 5min to turn them to tfrecords

In [7]:
%%capture
!unzip -Z1 "/content/drive/My Drive/high_res_anime_faces.zip" | head -5000 | sed 's| |\\ |g' | xargs unzip "/content/drive/My Drive/high_res_anime_faces.zip"

In [8]:
# resize each image
from PIL import Image
! mkdir /content/portraits_resized/

newsize = (64, 64)
img_dir = os.listdir('/content/portraits')
for filename in img_dir:
    path_filename = os.path.join('/content/portraits', filename)
    samp_image = Image.open(path_filename)
    samp_image = samp_image.resize(newsize)
    samp_image.save('/content/portraits_resized/' + filename)

FileNotFoundError: ignored

In [9]:
# convert images to tfrecord
# https://www.tensorflow.org/tutorials/load_data/tfrecord

# !python /content/stylegan2/dataset_tool.py create_from_images ./training_data ./portraits 

# However, create_from_images doesn't seem to work if not 512 x 512 ?
!python /content/stylegan2/dataset_tool.py create_from_images /content/training_data /content/portraits_resized 

# !python /content/stylegan2/dataset_tool.py create_from_images \
#     "/content/drive/My Drive/high_res_anime_faces_tfrecords" \
#     ./portraits

Traceback (most recent call last):
  File "/content/stylegan2/dataset_tool.py", line 16, in <module>
    import dnnlib.tflib as tflib
  File "/content/stylegan2/dnnlib/tflib/__init__.py", line 7, in <module>
    from . import autosummary
  File "/content/stylegan2/dnnlib/tflib/autosummary.py", line 30, in <module>
    from . import tfutil
  File "/content/stylegan2/dnnlib/tflib/tfutil.py", line 16, in <module>
    import tensorflow.contrib   # requires TensorFlow 1.x!
ModuleNotFoundError: No module named 'tensorflow.contrib'




---
# Loading pretrained model


https://www.reddit.com/r/MediaSynthesis/comments/gum6f1/tfdne_edit_furry_faces_inbrowser_google_colab/

In [ ]:
# import gdown
# !wget  -O /content/network-e621.pkl https://thisfursonadoesnotexist.com/model/network-e621-r-512-3194880.pkl

# !wget -O /content/nvidia-stylegan2-ada-ffhq-tf.pkl https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl 


--2022-05-05 01:36:34--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 13.224.160.5, 13.224.160.108, 13.224.160.125, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|13.224.160.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646055 (364M) [binary/octet-stream]
Saving to: ‘/content/nvidia-stylegan2-ada-ffhq-tf.pkl’

/content/nvidia-sty 100%[===================>] 363.97M  28.2MB/s    in 17s     

2022-05-05 01:36:52 (21.5 MB/s) - ‘/content/nvidia-stylegan2-ada-ffhq-tf.pkl’ saved [381646055/381646055]



In [ ]:
# tflib.init_tf()
# sess = tf.get_default_session()
# sess.list_devices()
# cores = tflex.get_cores()
# tflex.set_override_cores(cores)
# _G, _D, Gs = pickle.load(open("/content/nvidia-stylegan2-ada-ffhq-tf.pkl", "rb"))

In [ ]:
# import pretrained_networks
# network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
# _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
# # noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

In [ ]:
# Only if using NVlabs:
# get url from gdrive_urls in https://github.com/NVlabs/stylegan2/blob/master/pretrained_networks.py

# import gdown
# !wget -O /content/stylegan2-ffhq-config-f.pkl https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl 


In [ ]:
# only if using ZKTKZ repo:
import gdown
# !wget -O /content/stylegan2-ffhq-config-f.pkl http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl
!wget -O /content/stylegan2-horse-config-a.pkl http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-horse-config-a.pkl


--2022-05-06 15:09:52--  http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-horse-config-a.pkl
Resolving d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)... 18.65.227.151, 18.65.227.32, 18.65.227.14, ...
Connecting to d36zk2xti64re0.cloudfront.net (d36zk2xti64re0.cloudfront.net)|18.65.227.151|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302518494 (289M) [application/x-www-form-urlencoded]
Saving to: ‘/content/stylegan2-horse-config-a.pkl’

/content/stylegan2- 100%[===================>] 288.50M  29.1MB/s    in 14s     

2022-05-06 15:10:07 (20.8 MB/s) - ‘/content/stylegan2-horse-config-a.pkl’ saved [302518494/302518494]





---
# Training




https://colab.research.google.com/github/ZKTKZ/thdne/blob/master/StyleGAN2_Tazik_25GB_RAM.ipynb#scrollTo=vmeWpLdZdhq2

In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'

tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 12457290748965292404),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4667090122800505579),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 1235302875991361978),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 15964005991, 3302370878702370035)]

In [ ]:
# %cd training
# !python /content/stylegan2/training/training_loop.py --resume_pkl='gdrive:networks/stylegan2-ffhq-config-f.pkl'

Traceback (most recent call last):
  File "/content/stylegan2/training/training_loop.py", line 11, in <module>
    import dnnlib
ModuleNotFoundError: No module named 'dnnlib'


In [ ]:
# !python /content/stylegan2/run_training.py --num-gpus=1 --data-dir="/content/drive/My Drive/" \
# --config=config-f --dataset=high_res_anime_faces_tfrecords \
# --total-kimg=10000 --result-dir="/content/drive/My Drive/realfaces_anime/results/" 

# !python /content/stylegan2/run_training.py --num-gpus=1 --data-dir="/content/drive/My Drive/" \
# --config=config-a --dataset=high_res_anime_faces_tfrecords \
# --total-kimg=10000 --result-dir="/content/drive/My Drive/realfaces_anime/results/" \
# --resume-pkl='/content/stylegan2-horse-config-a.pkl'
# --resume-pkl='/content/stylegan2-ffhq-config-f.pkl'

!python /content/stylegan2/run_training.py --num-gpus=1 --data-dir=/content/stylegan2 --config=config-e --dataset=training_data \
--total-kimg=10000 --result-dir="/content/drive/My Drive/realfaces_anime/results/" 

# --spatial-augmentations=true --mirror-augment=true --metric=none 
# --min-h=4 --min-w=4 --res-log2=7 --lr=0.0005 

Local submit - run_dir: /content/drive/My Drive/realfaces_anime/results/00001-stylegan2-training_data-1gpu-config-e
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.

G                           Params    OutputShape       WeightShape     
---                         ---       ---               ---             
latents_in                  -         (?, 512)          -               
labels_in                   -         (?, 0)            -               
lod                         -         ()                -               
dlatent_avg                 -         (512,)            -               
G_mapping/latents_in        -         (?, 512)         

In [ ]:
%%capture
# 00002-stylegan2-training_data-1gpu-config-f
# network-snapshot-000000
!python /content/stylegan2/run_generator.py generate-images --seeds=0-100 --truncation-psi=0.7 \
--network='/content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/network-snapshot-000006.pkl'
#%cp -av /content/stylegan2/results/00000-generate-images /content/drive/'My Drive'/twist_moe/seeds-1.0


In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import math
import moviepy.editor
from numpy import linalg

dir = "/content/drive/My Drive/twist_moe/results/"
training_expm = "00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/"
snapshot = "network-snapshot-000006.pkl"
network_path = dir + training_expm + snapshot

def main():
    tflib.init_tf()
    _G, _D, Gs = pickle.load(open(network_path, "rb"))

    rnd = np.random
    latents_a = rnd.randn(1, Gs.input_shape[1])
    latents_b = rnd.randn(1, Gs.input_shape[1])
    latents_c = rnd.randn(1, Gs.input_shape[1])

    def circ_generator(latents_interpolate):
        radius = 40.0

        latents_axis_x = (latents_a - latents_b).flatten() / linalg.norm(latents_a - latents_b)
        latents_axis_y = (latents_a - latents_c).flatten() / linalg.norm(latents_a - latents_c)

        latents_x = math.sin(math.pi * 2.0 * latents_interpolate) * radius
        latents_y = math.cos(math.pi * 2.0 * latents_interpolate) * radius

        latents = latents_a + latents_x * latents_axis_x + latents_y * latents_axis_y
        return latents

    def mse(x, y):
        return (np.square(x - y)).mean()

    def generate_from_generator_adaptive(gen_func):
        max_step = 1.0
        current_pos = 0.0

        change_min = 10.0
        change_max = 11.0

        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

        current_latent = gen_func(current_pos)
        current_image = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
        array_list = []

        video_length = 1.0
        while(current_pos < video_length):
            array_list.append(current_image)

            lower = current_pos
            upper = current_pos + max_step
            current_pos = (upper + lower) / 2.0

            current_latent = gen_func(current_pos)
            current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
            current_mse = mse(array_list[-1], current_image)

            while current_mse < change_min or current_mse > change_max:
                if current_mse < change_min:
                    lower = current_pos
                    current_pos = (upper + lower) / 2.0

                if current_mse > change_max:
                    upper = current_pos
                    current_pos = (upper + lower) / 2.0


                current_latent = gen_func(current_pos)
                current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
                current_mse = mse(array_list[-1], current_image)
            # print(current_pos, current_mse)
        return array_list

    frames = generate_from_generator_adaptive(circ_generator)
    frames = moviepy.editor.ImageSequenceClip(frames, fps=30)

    # Generate video.
    mp4_file = '/content/drive/My Drive/circular.mp4'
    mp4_codec = 'libx264'
    mp4_bitrate = '3M'
    mp4_fps = 20

    frames.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

if __name__ == "__main__":
    main()

[MoviePy] >>>> Building video /content/drive/My Drive/circular.mp4
[MoviePy] Writing video /content/drive/My Drive/circular.mp4


100%|██████████| 1368/1368 [00:14<00:00, 93.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/circular.mp4 



In [ ]:
#https://stackoverflow.com/a/57378660/8773953
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/content/drive/My Drive/circular.mp4')

from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")

     |████████████████████████████████| 57 kB 4.5 MB/s 
     |████████████████████████████████| 60 kB 8.8 MB/s 


In [ ]:
!zip -r /content/anime_faces_furry_pretr.zip /content/drive/My\ Drive/twist_moe

  adding: content/drive/My Drive/twist_moe/ (stored 0%)
  adding: content/drive/My Drive/twist_moe/results/ (stored 0%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/ (stored 0%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/submit_config.pkl (deflated 51%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/submit_config.txt (deflated 70%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/log.txt (deflated 79%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/reals.jpg (deflated 4%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/fakes_init.jpg (deflated 3%)
  adding: content/drive/My Drive/twist_moe/results/00